In [ ]:
from pyspark.sql import *
from pyspark.ml import *

from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml import Pipeline

from builtins import round

In [ ]:
# input_1 : Window_size = 5
train_folder = 'input_1/train'
valid_folder = 'input_1/valid'
num_trees = [10,100]
n_fold = 5
n_digits = 4
max_depth = [5,10]  # added according to Diane's reply in piazza


# input_2 : Window_size - 10
# train_folder = 'input_2/train'
# valid_folder = 'input_2/valid'
# num_trees = [10,100]
# n_fold = 5
# n_digits = 3
# max_depth = [5,10]  # added according to Diane's reply in piazza


In [ ]:
ss = SparkSession.builder.config('spark.driver.extraClassPath', 'postgresql-42.2.18.jar')\  # no need jar actually
    .config("spark.executor.memory", "12g")\
    .config("spark.driver.memory", "12g")\
    .config("spark.executor.cores", 6)\
    .config('spark.executor.instances', 5).getOrCreate()

# QUESTION 1

In [ ]:
train_df = ss.read.parquet(train_folder).repartition(8).cache()
valid_df = ss.read.parquet(valid_folder).repartition(8).cache()

In [ ]:
print(train_df.count())

In [ ]:
print(valid_df.count())

# QUESTION 2

In [ ]:
# train_df.show(4)

In [ ]:
rf = RandomForestClassifier()
evaluator = BinaryClassificationEvaluator()  # areaUnderROC is default  https://spark.apache.org/docs/latest/api/java/index.html?org/apache/spark/ml/evaluation/BinaryClassificationEvaluator.html

paraGrid = ParamGridBuilder().addGrid(rf.numTrees, num_trees).build()

cv = CrossValidator(estimator = rf,
                   evaluator = evaluator,
                   numFolds = n_fold,
                   estimatorParamMaps = paraGrid)

cvmodel = cv.fit(train_df)

In [ ]:
rfpredicts = cvmodel.bestModel.transform(valid_df)

In [ ]:
print('RandomForestClassifier')

In [ ]:
print(cvmodel.bestModel.getNumTrees)

In [ ]:
print(round(evaluator.evaluate(rfpredicts),n_digits))

# QUESTION 3

In [ ]:
GBT = GBTClassifier()
evaluator = BinaryClassificationEvaluator()  # areaUnderROC is default

paramGrid = ParamGridBuilder().addGrid(GBT.maxDepth, max_depth).build()

cv = CrossValidator(estimator = GBT,
                   evaluator = evaluator,
                   numFolds = n_fold,
                   estimatorParamMaps = paramGrid)

cvmodel = cv.fit(train_df)

In [ ]:
GBTpredicts = cvmodel.bestModel.transform(valid_df)

In [ ]:
print('GBTClassifier')

In [ ]:
print(cvmodel.bestModel.getMaxDepth())

In [ ]:
print(round(evaluator.evaluate(GBTpredicts),n_digits))